In [ ]:
!pip install -q accelerate -U
!pip install -q bitsandbytes -U
!pip install -q trl -U
!pip install -q peft -U
!pip install -q transformers -U
!pip install -q huggingface_hub -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 21.6 MB/s eta 0:00:00
     

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Path to your desired folder in Google Drive
folder_path = '/content/drive/My Drive/Hugging Face - Fine Tuning LLM'

# Change directory
os.chdir(folder_path)

# Verify the current working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: /content/drive/My Drive/Hugging Face - Fine Tuning LLM


In [ ]:
import csv

# Define the input CSV file path
input_csv = 'chennai_q_and_a.csv'

# Read the CSV file and print the headers
with open(input_csv, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    print(csv_reader.fieldnames)

['\ufeffQuestions', 'Answers']


In [ ]:
import csv
import json
#Questions,Answers
# Define the input and output file paths
input_csv = 'chennai_q_and_a.csv'
output_json = 'chennai_q_and_a.json'

# Read the CSV file
with open(input_csv, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    data = []

    # Convert each row to the desired format
    for row in csv_reader:
        example = {
            "messages": [
                {"role": "user", "content": row['\ufeffQuestions']},
                {"role": "assistant", "content": row['Answers']}
            ]
        }
        data.append(example)

# Write the converted data to a JSON file
with open(output_json, mode='w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print(f"Conversion complete. Data saved to {output_json}")

Conversion complete. Data saved to chennai_q_and_a.json


In [ ]:
import json

# Load the converted JSON file
with open(output_json, mode='r', encoding='utf-8') as json_file:
    converted_data = json.load(json_file)

# Print a few examples to validate
for example in converted_data[:5]:
    print(json.dumps(example, indent=4))

{
    "messages": [
        {
            "role": "user",
            "content": "What is the URL for Mahabalipuram?"
        },
        {
            "role": "assistant",
            "content": "https://www.google.com/maps/place/Mahabalipuram"
        }
    ]
}
{
    "messages": [
        {
            "role": "user",
            "content": "What are the coordinates for Mahabalipuram?"
        },
        {
            "role": "assistant",
            "content": "Latitude: 12.6223681, Longitude: 80.1705427"
        }
    ]
}
{
    "messages": [
        {
            "role": "user",
            "content": "What is the description of Mahabalipuram?"
        },
        {
            "role": "assistant",
            "content": "An ancient historic town and a UNESCO World Heritage site, famous for its rock-cut temples and monuments, including the Shore Temple and the Five Rathas. It's known for its architectural and sculptural splendor dating back to the Pallava dynasty."
        }
    ]
}


In [ ]:
from datasets import load_dataset

# Load the converted dataset
dataset = load_dataset('json', data_files=output_json)

# Print the dataset structure
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 1252
    })
})


In [ ]:
from transformers import AutoTokenizer

template_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
def format_prompt(example):
  """Format the prompt using the <|user|> and <|assistant|> format"""
  chat = example['messages']
  prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)

  return {'text': prompt}

# print(format_prompt(dataset[0])['text'])
dataset = dataset.map(format_prompt)

Map:   0%|          | 0/1252 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['messages', 'text'],
        num_rows: 1252
    })
})


In [ ]:
print(dataset['train'][1])

{'messages': [{'content': 'What are the coordinates for Mahabalipuram?', 'role': 'user'}, {'content': 'Latitude: 12.6223681, Longitude: 80.1705427', 'role': 'assistant'}], 'text': '<|user|>\nWhat are the coordinates for Mahabalipuram?</s>\n<|assistant|>\nLatitude: 12.6223681, Longitude: 80.1705427</s>\n'}


In [ ]:
from transformers import pipeline

# Base model
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

# Initialize the pipeline
pipe = pipeline(task='text-generation', model=model_name, device='cuda')

# Test the pipeline
result = pipe("What is the capital of France?")
print(result)


config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda


[{'generated_text': 'What is the capital of France?\nWhat is the capital of France?\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital of France? The capital of France is Paris.\nWhat is the capital

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
# do the  4-bit quantization configuration in Q-LORA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=True
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_size="left"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    quantization_config=bnb_config
)

In [ ]:
model.config.use_cache=False
model.config.pretraining_tp=1

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
)

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)

In [ ]:
print(dataset.column_names)

{'train': ['messages', 'text']}


In [ ]:
train_dataset = dataset['train']

In [ ]:
train_dataset = train_dataset.rename_column("messages", "train")

In [ ]:
print(train_dataset['train'][1])

[{'content': 'What are the coordinates for Mahabalipuram?', 'role': 'user'}, {'content': 'Latitude: 12.6223681, Longitude: 80.1705427', 'role': 'assistant'}]


In [ ]:
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer

output_dir = "train_dir"

args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=10,
    logging_steps=150,
    fp16=True,
    gradient_checkpointing=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    dataset_text_field='text',  # Ensure this matches the column name in your dataset
    tokenizer=tokenizer,
    args=args,
    max_seq_length=512,
    peft_config=peft_config
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1252 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
150,3.307800
300,2.001000
450,1.737700
600,1.545800
750,1.379700
900,1.259800
1050,1.111900
1200,0.944500
1350,0.819300
1500,0.773200


TrainOutput(global_step=1560, training_loss=1.4598136437244904, metrics={'train_runtime': 2795.3963, 'train_samples_per_second': 4.479, 'train_steps_per_second': 0.558, 'total_flos': 5040173727424512.0, 'train_loss': 1.4598136437244904, 'epoch': 9.939297124600639})

In [ ]:
trainer.model.save_pretrained("TinyLlama-1.1B-qlora-chennai-q-and-a")

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora-chennai-q-and-a",
    device_map='auto'
)

merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: Can't find 'adapter_config.json' at 'TinyLlama-1.1B-qlora-chennai-q-and-a'

In [ ]:
from transformers import pipeline

prompt = """<|user|>
Can you get me the location of Mahabalipuram</s>
<|assistant|>
"""

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_size="left"

pipe = pipeline(task='text-generation', model=merged_model, tokenizer=tokenizer)
output = pipe(prompt)
print(output[0]['generated_text'])

Device set to use cuda:0


<|user|>
Can you get me the location of Mahabalipuram</s>
<|assistant|>
https://www.google.com/maps/place/Mahabalipuram


In [ ]:
!zip -r tiny_llama_qlora_adapter.zip TinyLlama-1.1B-qlora-chennai-q-and-a

  adding: TinyLlama-1.1B-qlora-chennai-q-and-a/ (stored 0%)
  adding: TinyLlama-1.1B-qlora-chennai-q-and-a/README.md (deflated 66%)
  adding: TinyLlama-1.1B-qlora-chennai-q-and-a/adapter_model.safetensors (deflated 8%)
  adding: TinyLlama-1.1B-qlora-chennai-q-and-a/adapter_config.json (deflated 55%)
